In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path
from scripts.cross_match_scripts import my_scaler_forward
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns', 500)
import seaborn as sns
import numpy as np
%matplotlib inline

matplotlib settings set


matplotlib settings set


# Catalogs loading and preprocessing

In [3]:
photo_cat = pd.read_pickle(data_path+'csc_allsky_desi_r30_gaia_dered_training.gz_pkl', compression='gzip')
x_ray_flux_lim = 3e-15
photo_cat = photo_cat.query('(csc_flux_05_2 > @x_ray_flux_lim) | is_counterpart == False')
photo_cat = photo_cat.query('csc_dec>32') #ignore DECALS data because LH area is within MzLS+BASS footprint, see https://www.legacysurvey.org/status/
photo_cat = photo_cat.replace([np.inf, -np.inf], np.nan)
photo_cat.sample(5)

,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,r_false_003_dens_10_30,num_srcs_sep_less_r_false,is_counterpart
1198930,2CXO J142038.8+530816,215.161868,53.137900,NaN,0.532789,NaN,9011,1770,596103,215.152873,53.138088,1,0.305182,1.547190,5.376481,16.892649,7.865379,51.933456,-145.504150,591.86110,203.579500,69.369010,4.614617,1.445622,0.002151,0.000025,23.757217,22.005001,20.661930,19.428959,20.259598,18.211147,NaN,7.424538,22.075510,44.779663,36.288210,9.456862,2.408785,-0.733068,0.0,0.0,0.0,0.0,0.0,0.0,19.439262,9011_596103_1770,23.788603,23.788603,23.757217,22.026141,22.026141,22.005001,20.673755,20.673755,20.661930,19.430756,19.430756,19.428959,16.731756,20.260701,20.260701,20.259598,16.921701,18.211382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.074338e-15,1.752216,3.095287,1.343071,1.232971,1.745403,-0.830639,NaN,NaN,NaN,1.163504,1,False
1186018,2CXO J141533.9+361337,213.891661,36.227036,NaN,0.862115,NaN,9011,2820,526619,213.893781,36.233998,1,5.652381,10.842139,0.574984,-134.422410,-79.166306,-636.480400,-1430.210100,5.31466,2.836205,0.996766,0.601664,0.178679,0.000628,0.000014,20.586985,19.890362,23.088638,NaN,NaN,NaN,NaN,13.030744,18.259285,0.574054,-104.267460,-33.463913,-15.953089,-5.279218,0.0,0.0,0.0,0.0,0.0,0.0,25.807626,9011_526619_2820,20.619421,20.619421,20.586985,19.912213,19.912213,19.890362,23.100861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.696623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.015249,1,False
606940,2CXO J080318.6+325825,120.827859,32.973611,NaN,0.997459,NaN,9011,391,510872,120.820400,32.970495,1,0.079325,0.313402,0.426516,3.889194,2.948273,246.576750,-6149.847700,650.78740,301.180820,188.613280,3.009752,0.675834,0.000749,0.000010,25.061136,23.631529,23.353443,21.014454,21.319390,16.518692,NaN,2.023615,5.438955,5.857622,6.747221,2.423749,6.748521,-19.402029,0.0,0.0,0.0,0.0,0.0,0.0,25.165026,9011_510872_391,25.251475,NaN,NaN,23.759746,23.759746,23.631529,23.425162,23.425162,23.353443,21.025351,21.025351,21.014454,18.326351,21.326081,NaN,NaN,NaN,16.520120,16.520120,16.518692,11.346120,NaN,NaN,NaN,NaN,7.606935e-16,NaN,NaN,0.278086,2.338989,NaN,NaN,6.834751,NaN,NaN,0.901247,1,False
867883,2CXO J111825.6+402423,169.607029,40.406603,NaN,0.493863,NaN,9011,1418,545715,169.605050,40.414230,1,0.832617,1.565817,1.931876,-0.627328,2.354103,15.809086,-73.579155,773.36820,202.538160,200.131530,4.136941,0.904705,0.000987,0.000012,22.646723,21.978008,21.765398,NaN,21.568602,19.502342,NaN,23.154665,22.284061,27.329832,-1.275952,2.239128,0.496787,-0.256486,0.0,0.0,0.0,0.0,0.0,0.0,27.989624,9011_545715_1418,22.698887,22.698887,22.646723,22.013147,22.013147,21.978008,21.785052,21.785052,21.765398,NaN,NaN,NaN,NaN,21.570436,NaN,NaN,NaN,19.502733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.284186e-15,0.668715,0.881325,0.212610,NaN,NaN,NaN,NaN,NaN,NaN,1.561005,1,False
1425845,2CXO J171935.2+353626,259.896860,35.607340,NaN,0.996004,NaN,9011,4682,523260,259.889052,35.607354,1,7.238544,14.145522,25.123878,32.560880,31.474083,411.789760,-51.509796,262.04153,89.541050,34.537020,3.495767,1.052318,0.001656,0.000021,20.240911,19.549381,18.958351,18.711964,18.751251,15.9

In [4]:
print('fraction of missed measurements per is_counterpart per filter, in per cent:')
photo_cat.groupby('is_counterpart')[['rel_dered_mag_g',	'rel_dered_mag_r',	'rel_dered_mag_z',	'rel_dered_mag_w1',	'rel_dered_mag_w2',	'rel_dered_mag_w3',	'rel_dered_mag_w4']].agg(lambda x: np.round(100*np.mean(np.
isnan(x)),1))

fraction of missed measurements per is_counterpart per filter, in per cent:


,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_mag_w3,rel_dered_mag_w4
is_counterpart,,,,,,,
False,26.7,20.7,12.5,44.0,65.7,95.0,90.7
True,8.8,5.9,1.3,3.1,6.1,67.6,81.3


In [5]:
features_cols = ['rel_dered_mag_g','rel_dered_mag_r','rel_dered_mag_z'] 
features_cols += ['rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z']


features_cols += ['rel_dered_mag_w1', 'rel_dered_mag_w2']
features_cols += ['rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2']

features_cols += ['rel_dered_mag_w3', 'rel_dered_mag_w4']
features_cols += ['rel_dered_z_w3', 'rel_dered_r_w4', 'rel_dered_w3_w4']


target_col = ['is_counterpart']

photo_cat = photo_cat[features_cols+target_col+['csc_flux_05_2']]

#photo_cat= photo_cat.dropna(subset = features_cols, how = 'any')

print(photo_cat.is_counterpart.value_counts())
photo_cat = photo_cat.sample(frac=1)
photo_cat.reset_index(inplace=True, drop = True)

photo_cat_scaled = my_scaler_forward(photo_cat)
photo_cat_scaled.to_pickle('./model/train_test_validation_data.pkl')
photo_cat_scaled.head(15)


False    207917
True      12459
Name: is_counterpart, dtype: int64
rel_dered_mag_g scaled by 1/35
rel_dered_mag_r scaled by 1/35
rel_dered_mag_z scaled by 1/35
rel_dered_g_r scaled by 1/10
rel_dered_r_z scaled by 1/10
rel_dered_g_z scaled by 1/10
rel_dered_mag_w1 scaled by 1/35
rel_dered_mag_w2 scaled by 1/35
rel_dered_z_w1 scaled by 1/10
rel_dered_r_w2 scaled by 1/10
rel_dered_w1_w2 scaled by 1/10
rel_dered_mag_w3 scaled by 1/35
rel_dered_mag_w4 scaled by 1/35
rel_dered_z_w3 scaled by 1/10
rel_dered_r_w4 scaled by 1/10
rel_dered_w3_w4 scaled by 1/10


,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_g_r,rel_dered_r_z,rel_dered_g_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_mag_w3,rel_dered_mag_w4,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,is_counterpart,csc_flux_05_2
0,0.653636,0.649197,0.620585,0.015535,0.100144,0.115679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
1,0.666468,0.627255,0.613139,0.137244,0.049406,0.186651,0.581737,0.589558,0.109907,0.131938,-0.027376,NaN,NaN,NaN,NaN,NaN,False,NaN
2,0.692168,0.642519,0.604599,0.173772,0.132722,0.306494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,0.692335,0.666855,0.640142,0.089178,0.093496,0.182675,0.601343,NaN,0.135795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
4,0.675167,0.670021,NaN,0.018011,NaN,NaN,NaN,0.601151,NaN,0.241048,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
5,0.664870,0.623429,0.584599,0.145042,0.135907,0.280948,0.593579,0.599755,-0.031429,0.082861,-0.021617,NaN,NaN,NaN,NaN,NaN,False,NaN
6,0.657043,0.624838,0.607742,0.112716,0.059836,0.172552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
7,0.691313,0.666578,0.660874,0.086572,0.019963,0.106535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
8,NaN,0.657307,0.604870,NaN,0.183529,NaN,0.563878,0.585380,0.143472,0.251746,-0.075255,NaN,NaN,NaN,NaN,NaN,False,NaN
9,0.692100,0.672094,0.652010,0.070020,0.070295,0.140315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
